# Python爬虫实战
## 基本概念
### 开发者工具
- Elements：允许用户从浏览器的角度来观察网页，用户可以借此看到Chrome渲染页面所需要的HTML、CSS和DOM（Document Object Model）对象。
- Network：可以看到网页向服务气请求了哪些资源、资源的大小以及加载资源的相关信息。此外，还可以查看HTTP的请求头、返回内容等。
- Source：即源代码面板，主要用来调试JavaScript。
- Console：即控制台面板，可以显示各种警告与错误信息。在开发期间，可以使用控制台面板记录诊断信息，或者使用它作为shell在页面上与JavaScript交互。
- Performance：使用这个模块可以记录和查看网站生命周期内发生的各种事情来提高页面运行时的性能。
- Memory：这个面板可以提供比Performance更多的信息，比如跟踪内存泄漏。
- Application：检查加载的所有资源。
- Security：即安全面板，可以用来处理证书问题等。

## 实际应用
### 爬取python之禅(GET) 


In [4]:
import requests
url = 'https://www.python.org/dev/peps/pep-0020/'
res = requests.get(url)
text = res.text

利用python内置find函数定位python之禅

1. 通过审查元素，使用快捷键Ctrl+shift+c快速定位到这段话，也可以发现这段话包围
在pre标签中`<pre class="literal-block">`
2. `<pre>`标签可定义预格式化的文本。被包围在`<pre>`标签元素中的文本通常会保留空格和换行符。而文本也会呈现为等宽字体。

find函数返回所查找字符串**开始的索引值**，`text.find('<pre')+28`中的数字28是为了去除这一行`<pre class="literal-block">`

In [42]:
print(text.find('<pre'))
print(text.find('</pre>'))
print(text.find('<pre')+28)
print(text.find('</pre>')-1)
print(text[text.find('<pre'):text.find('</pre>')])
print(text[text.find('<pre')+28 : text.find('</pre>')-1])

23529
24380
23557
24379
<pre class="literal-block">
Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than compl

In [12]:
with open('zone_of_python.txt', 'w') as f:
    f.write(text[text.find('<pre') + 28:text.find('</pre') -1])

利用python自带的urllib完成以上操作：

In [13]:
import urllib
url = 'https://www.python.org/dev/peps/pep-0020/'
res = urllib.request.urlopen(url).read().decode('utf-8')
print(res[res.find('<pre')+28:res.find('</pre>')-1])

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### 爬取金山词霸（POST）
1. 首先进入金山词霸首页http://www.iciba.com/
2. 然后打开开发者工具下的“Network”，翻译一段话，比如刚刚我们爬到的第一句话“Beautiful is better than ugly.”
3. 点击翻译后可以发现Name下多了一项请求方法是POST的数据，文件名为`ajax.php?a=fy`，点击Preview可以发现数据中有我们想要的翻译结果
4. 我们目前需要用到的两部分信息是Request Headers中的User-Agent和From Data

**注意点：**

向接口发送一次翻译请求后，中间睡眠3s，然后再发送请求，否则会被远程主机强制关闭。


In [21]:
import requests
import time


def translate(word):
    url = "http://fy.iciba.com/ajax.php?a=fy"  # 网页审查元素中可知
    data = {
        'f':'auto',
        't':'auto',
        'w':word,
    }
    headers = {
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'       
    }
    response = requests.post(url, data=data, headers=headers)
    json_data = response.json()
    return json_data


def run(word):
    result = translate(word)['content']['out']  # 从preview中可知
    print(result)
    return result


def main():
    with open('zone_of_python.txt') as f:
        zh = []
        for word in f:
            zh.append(run(word))
            time.sleep(3)
            

    with open('zone_of_python_zh-CN.txt', 'w') as g:
        for i in zh:
            g.write(i + '\n')


if __name__ == '__main__':
    main()

美丽胜过丑陋。
 外显优于内隐..
 简单胜于复杂。
 复杂胜于复杂。
 平比嵌套好..
 疏而不密..
可读性计数。
 特殊情况不足以打破规则。
 尽管实用性胜过纯度。
 错误永远不应该悄悄地过去。
 除非有明确的沉默。
 面对暧昧，拒绝猜测的诱惑..
 应该有一种----最好只有一种----明显的办法来做到这一点。
 虽然这种方式一开始可能不明显，除非你是荷兰人。
 现在总比永远好。
 虽然从来没有比现在更好。
 如果实施很难解释，那是个坏主意。
 如果实现很容易解释，这可能是个好主意。
 命名空间是一个伟大的想法-让我们做更多的这些！


### 爬取豆瓣电影（GET进阶）
![](https://raw.githubusercontent.com/guangmujun/SpiderLearning/master/imgs/1.jpg)

In [1]:
import requests
import os

if not os.path.exists('image'):
    os.mkdir('image')


def parse_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
    }
    res = requests.get(url, headers=headers)
    text = res.text
    item = []
    for i in range(25):
        text = text[text.find('alt')+3:]
        item.append(extract(text))
    return item


def extract(text):
    text = text.split('"')
    name = text[1]
    image = text[3]
    return name, image


def write_movies_file(item, stars):
    print(item)
    # 保存电影信息
    with open('douban_file.txt', 'a', encoding='utf-8') as f:
        f.write('排名: %d\t电影名： %s\n' %(stars, item[0]))
    
    # 保存图片
    r = requests.get(item[1])
    with open('image/' + str(item[0]) + '.jpg', 'wb') as f:
        f.write(r.content)


def main():
    stars = 1
    for offset in range(0, 250, 25):  # 每页25部电影
        url = 'https://movie.douban.com/top250?start=' + str(offset) + '&filter='
        for item in parse_html(url):
            write_movies_file(item, stars)
            stars += 1


if __name__ == '__main__':
    main()

r/public/p616779645.jpg')
('当幸福来敲门', 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p1312700628.jpg')
('怦然心动', 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p501177648.jpg')
('触不可及', 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p1454261925.jpg')
('蝙蝠侠：黑暗骑士', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p462657443.jpg')
('控方证人', 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p1505392928.jpg')
('活着', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2513253791.jpg')
('乱世佳人', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p1963126880.jpg')
('寻梦环游记', 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2503997609.jpg')
('末代皇帝', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p452089833.jpg')
('摔跤吧！爸爸', 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2457983084.jpg')
('指环王3：王者无敌', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p1910825503.jpg')
('少年派的奇幻

## 使用API
所谓的采集网络数据，并不一定必须从网页中抓取数据，而api（Application Programming Iterface）的用处就在这里：API为开发者提供了方便友好的接口，不同的开发者用不同的语言都能获取相同的数据。目前API一般会以XML（Extensible Markup Language，可拓展标记语言）或者JSON（JavaScript Object Notation）格式来返回服务器响应，其中JSON数据格式越来越受到人们的欢迎，我们后面的课程也会详细介绍JSON格式。

## 百度地图API使用
1. 创建百度API应用的地址：http://lbsyun.baidu.com/apiconsole/key 
2. 注意应用类型要选择浏览器端
3. 官方说明文档：http://lbsyun.baidu.com/index.php?title=jspopular3.0

yield:带yield的函数是一个生成器，而不是一个函数了，这个生成器有一个函数就是next函数

eval:用来执行一个字符串表达式，并返回表达式的值。

In [39]:
# 实现地理编码
import requests

def getUrl(*address):
    ak = 'FM3x9wRFVvwmZIgnCFFG9ysPQxnmpkR3'
    if len(address) < 1:
        return None
    else:
        for add in address:
            url = 'http://api.map.baidu.com/geocoding/v3/?address={0}&output=json&ak={1}'.format(add,ak)  
            yield url


def getPosition(url):
    res = requests.get(url)
    json_data = eval(res.text)
    if json_data['status'] == 0:
        lat = json_data['result']['location']['lat'] #纬度
        lng = json_data['result']['location']['lng'] #经度
    else:
        print("Error output!")
        return json_data['status']
    return lat,lng


if __name__ == '__main__':
    address = ['北京市清华大学','北京市北京大学','保定市华北电力大学','上海市复旦大学','武汉市武汉大学']
    for add in address:
        add_url = list(getUrl(add))[0]
        print('url:', add_url)
        try:
            lat, lng = getPosition(add_url)
            print("{0}|经度:{1}|纬度:{2}.".format(add,lng,lat))
        except BaseException as e:
            print(e)

url: http://api.map.baidu.com/geocoding/v3/?address=北京市清华大学&output=json&ak=FM3x9wRFVvwmZIgnCFFG9ysPQxnmpkR3
done
北京市清华大学|经度:116.33337396094367|纬度:40.009645090734296.
url: http://api.map.baidu.com/geocoding/v3/?address=北京市北京大学&output=json&ak=FM3x9wRFVvwmZIgnCFFG9ysPQxnmpkR3
done
北京市北京大学|经度:116.31683256328296|纬度:39.99887680537622.
url: http://api.map.baidu.com/geocoding/v3/?address=保定市华北电力大学&output=json&ak=FM3x9wRFVvwmZIgnCFFG9ysPQxnmpkR3
done
保定市华北电力大学|经度:115.52130317483764|纬度:38.89477430426888.
url: http://api.map.baidu.com/geocoding/v3/?address=上海市复旦大学&output=json&ak=FM3x9wRFVvwmZIgnCFFG9ysPQxnmpkR3
done
上海市复旦大学|经度:121.74295536914276|纬度:31.06665792321301.
url: http://api.map.baidu.com/geocoding/v3/?address=武汉市武汉大学&output=json&ak=FM3x9wRFVvwmZIgnCFFG9ysPQxnmpkR3
done
武汉市武汉大学|经度:114.37292090919235|纬度:30.543803317143624.


## JavaScript与AJAX技术
### 基础概念
1. 开发者通过程序获取到的HTTP响应内容都是原始的HTML数据，但浏览器中的页面其实是在HTML的基础上，经过JavaScript进一步加工和处理后生成的效果。
2. 今天，以AJAX（Asynchronous JavaScript and XML，异步JavaScript与XML）技术为代表的结合JavaScript、CSS、HTML等语言的网页开发技术已经成为绝对的主流。
3. JavaScript语言一般被定义为一种“面向对象、动态类型的解释性语言”，最初由Netscape公司为Navigator浏览器开发，目的是作为新一代浏览器的脚本语言支持。
4. 对于今天任何一个正式的网站页面而言，HTML决定了网页的基本内容，CSS（Cascading Style Sheets，层叠样式表）描述了网页的样式布局，JavaScript 则控制了用户与网页的交互。
### JS语言特点
- 动态语言

动态语言是指程序在运行时可以改变其结构：新的函数可以被引进，已有的函数可以被删除等在结构上的变化。JavaScript便是一个动态语言。除此之外如Ruby、Python等也都属于动态语言，而C、C++等语言则不属于动态语言。比如在JavaScript中可以在对象定义之后动态的为其添加属性和方法

- 脚本语言

脚本语言是为了缩短传统的编写-编译-链接-运行（edit-compile-link-run）过程而创建的计算机编程语言，只在被调用时进行解释或编译，然后执行。它的命名起源于一个脚本“screenplay”，每次运行都会使对话框逐字重复。早期的脚本语言经常被称为批量处理语言或工作控制语言。

- 弱类型

弱/强类型指的是语言类型系统的类型检查的严格程度，弱类型的语言在声明变量的时候不必进行变量类型的确定，语言的运行时会隐式做数据类型转换，对于弱类型语言来说，不同类型的变量可以进行直接运算，而强类型的则不可以。

### 基本语法
- JavaScript的执行顺序：按照HTML文件中出现的顺序依次执行
- 大小写敏感：JavaScript严格区分大小写
- 忽略空白符和换行符
- 语句分隔符：使用；结束语句，可以把多个语句写在一行，最后一个语句的分号可以省略，但尽量不要省略。可以使用{}括成一个语句组，形成一个block
- 通过\对代码进行折行操作：document.write(‘hello\world’);
- //单行注释  多行注释/注释内容/
- JavaScript中的保留字：abstract，else，instanceof，super，boolean，enum，int，switch，break，export，interface，synchronized，byte，extends，let，this，case，false，long，throw，catch，final，native，throws，char，finally，new，transient，class，float，null，true，const，for，package，try，continue，function，private，typeof，debugger，goto，protected，var，defaut，if，public，void，delete，inplements，return，volatile，do，import，short，while，doble，in，static，width
- 通过document.write()向文档书写内容
- 通过console.log()向控制台写入内容
- 语法错误：通过控制台进行调试
- 逻辑错误：通过alert() 进行调试

### AJAX技术

AJAX技术与其说是一种“技术”，不如说是一种“方案”。如上文所述，在网页中使用JavaScript 加载页面中数据的过程，都可以看作AJAX技术。AJAX技术改变了过去用户浏览网站时一个请求对应一个页面的模式，允许浏览器通过异步请求来获取数据，从而使得一个页面能够呈现并容纳更多的内容，同时也就意味着更多的功能。只要用户使用的是主流的浏览器，同时允许浏览器执行JavaScript，用户就能够享受网页中的AJAX内容。

基于Python编写的爬虫程序可以做出两种改进

1. 一种是通过分析AJAX内容（需要开发者手动观察和实验），观察其请求目标、请求内容和请求的参数等信息，最终编写程序来模拟这样的JavaScript 请求，从而获取信息（这个过程也可以叫作“逆向工程”）
2. 另外一种方式则比较取巧，那就是直接模拟出浏览器环境，使得程序得以通过浏览器模拟工具“移花接木”，最终通过浏览器渲染后的页面来获得信息。这两种方式的选择与JavaScript在网页中的具体使用方法有关。

## 问题记录
Q1：爬取python之禅的应用中，这句什么意思？
```python
print(text[text.find('<pre')+28:text.find('</pre>')-1])
```
A1： `text.find('<pre')+28`是一个索引值，`text.find('</pre>')-1`也是一个索引值，从字符串中提取子字符串。